In [1]:
#set adequate flag for Theano on lxplus
import theano
theano.config.gcc.cxxflags = '-march=corei7'

#load needed things
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from keras.models import Sequential, Model
from keras.optimizers import SGD
from keras.layers import Input, Activation, Dense, Convolution2D, MaxPooling2D, Dropout, Flatten
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pyp
import ROOT
import itertools
import math

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

Using Theano backend.


Welcome to JupyROOT 6.07/07


In [2]:
#format the inputs from TTree
def formatInputs(files, njets, min_m4l, max_m4l):    
    formated_inputs = {}
    formated_inputs['VBF'] = []
    formated_inputs['GGH'] = []
    formated_inputs['ttH'] = []
    formated_inputs['ZH'] = []
    formated_inputs['WH'] = []
    formated_inputs['ggZZ'] = []
    formated_inputs['qqZZ'] = []
    formated_inputs['TTWJets'] = []
    formated_inputs['TTZ'] = []
    formated_inputs['ZZZ'] = []
    formated_inputs['WWZ'] = []
    formated_inputs['WZZ'] = []
    formated_inputs['WZ'] = []
    formated_inputs['WW'] = []
    formated_inputs['DYJets'] = []
    formated_inputs['TTbar'] = []
    formated_inputs['WJets'] = []

    for ifile in range(len(files)):
        file_name = files[ifile]
        tfile = ROOT.TFile.Open(file_name)
        tree = tfile.Get('HZZ4LeptonsAnalysisReduced')
        
        nacepted = 0
        for ievt, evt in enumerate(tree):
            if(evt.f_njets_pass >= njets and evt.f_mass4l >= min_m4l and evt.f_mass4l <= max_m4l): #VBF region
                event = []
                        
                lep = []
                lep.append( evt.f_lept1_pt )
                lep.append( evt.f_lept1_eta )
                lep.append( evt.f_lept1_phi )
                lep.append( evt.f_lept1_pdgid )
                event.append(lep)

                lep = []
                lep.append( evt.f_lept2_pt )
                lep.append( evt.f_lept2_eta )
                lep.append( evt.f_lept2_phi )
                lep.append( evt.f_lept2_pdgid )
                event.append(lep)

                lep = []
                lep.append( evt.f_lept3_pt )
                lep.append( evt.f_lept3_eta )
                lep.append( evt.f_lept3_phi )
                lep.append( evt.f_lept3_pdgid )
                event.append(lep)
    
                lep = []
                lep.append( evt.f_lept4_pt )
                lep.append( evt.f_lept4_eta )
                lep.append( evt.f_lept4_phi )
                lep.append( evt.f_lept4_pdgid )
                event.append(lep)
            
                jet = []
                jet.append( evt.f_jet1_highpt_pt )
                jet.append( evt.f_jet1_highpt_eta )
                jet.append( evt.f_jet1_highpt_phi )
                jet.append( evt.f_jet1_highpt_e*(ROOT.TMath.CosH(evt.f_jet1_highpt_eta)) )
                event.append(jet)

                jet = []
                jet.append( evt.f_jet2_highpt_pt )
                jet.append( evt.f_jet2_highpt_eta )
                jet.append( evt.f_jet2_highpt_phi )
                jet.append( evt.f_jet2_highpt_e*(ROOT.TMath.CosH(evt.f_jet2_highpt_eta)) )
                event.append(jet)

                jet = []
                jet.append( evt.f_jet3_highpt_pt )
                jet.append( evt.f_jet3_highpt_eta )
                jet.append( evt.f_jet3_highpt_phi )
                jet.append( evt.f_jet3_highpt_e*(ROOT.TMath.CosH(evt.f_jet3_highpt_eta)) )
                event.append(jet)
                
                event.append(evt.f_D_jet) #CMS VBF discriminant
                event.append(evt.f_Djet_VAJHU) #MELA
                event.append(evt.f_weight) #event weight
                
                event.append(evt.f_mass4l)
                event.append(evt.f_Z1mass)
                event.append(evt.f_Z2mass)                
                #nacepted += 1
                
                
                if(file_name.find('output_VBF') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['VBF'].append(event)
                    nacepted += 1
                if(file_name.find('output_GluGluH') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['GGH'].append(event)
                    nacepted += 1
                if(file_name.find('output_ttH') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['ttH'].append(event)
                    nacepted += 1
                if(file_name.find('output_ZH') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['ZH'].append(event)
                    nacepted += 1
                if(file_name.find('output_WminusH') != -1 or file_name.find('output_WplusH') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['WH'].append(event)
                    nacepted += 1
                if(file_name.find('output_GluGluToContinToZZ') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['ggZZ'].append(event)
                    nacepted += 1
                if(file_name.find('output_ZZTo4L') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['qqZZ'].append(event)
                    nacepted += 1
                if(file_name.find('output_TTWJets') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['TTWJets'].append(event)
                    nacepted += 1
                if(file_name.find('output_TTZ') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['TTZ'].append(event)
                    nacepted += 1
                if(file_name.find('output_ZZZ') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['ZZZ'].append(event)
                    nacepted += 1
                if(file_name.find('output_WWZ') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['WWZ'].append(event)
                    nacepted += 1
                if(file_name.find('output_WZZ') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['WZZ'].append(event)
                    nacepted += 1
                if(file_name.find('output_WZ') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['WZ'].append(event)
                    nacepted += 1
                if(file_name.find('output_WW') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['WW'].append(event)
                    nacepted += 1
                if(file_name.find('output_DYJets') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['DYJets'].append(event)
                    nacepted += 1
                if(file_name.find('output_TTTo') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['TTbar'].append(event)
                    nacepted += 1                            
                if(file_name.find('output_WJets') != -1):
                    if(nacepted == 0):
                        print ('Processing', file_name)
                    formated_inputs['WJets'].append(event)
                    nacepted += 1


        #print ('%i - Processed (%i)' % (ifile, nacepted)), file_name

    return formated_inputs

In [3]:
#loads input data
file_names2e2mu = open('/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos2e2mu_25ns/filelist_2e2mu_2016_Spring16_AN_Bari_miniaod_mc.txt','r')
file_names4e = open('/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos4e_25ns/filelist_4e_2016_Spring16_AN_Bari_miniaod_mc.txt','r')
file_names4mu = open('/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos4mu_25ns/filelist_4mu_2016_Spring16_AN_Bari_miniaod_mc.txt','r')
path = '/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/'

files_addresses = []
for i in file_names2e2mu.readlines():
    files_addresses.append( path+'histos2e2mu_25ns/'+i.rstrip() )
for i in file_names4e.readlines():
    files_addresses.append( path+'histos4e_25ns/'+i.rstrip()  )
for i in file_names4mu.readlines():
    files_addresses.append( path+'histos4mu_25ns/'+i.rstrip()  )

#print files_addresses
events = formatInputs(files_addresses, 2, 118, 130)

('Processing', '/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos2e2mu_25ns/output_GluGluHToZZTo4L_M125_13TeV_powheg2_JHUgenV6_pythia8.root')
('Processing', '/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos2e2mu_25ns/output_ttH_HToZZ_4LFilter_M125_13TeV_powheg2_JHUgenV6_pythia8.root')
('Processing', '/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos2e2mu_25ns/output_VBF_HToZZTo4L_M125_13TeV_powheg2_JHUgenV6_pythia8.root')
('Processing', '/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos2e2mu_25ns/output_ZH_HToZZ_4LFilter_M125_13TeV_powheg2-minlo-HZJ_JHUgenV6_pythia8.root')
('Processing', '/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos2e2mu_25ns/output_WminusH_HToZZTo4L_M125_13TeV_powheg2-minlo-HWJ_JHUgenV6_pythia8.root')
('Processing', '/afs/cern.ch/work/m/mmelodea/private/Higgs_Ntuples/files2016/histos2e2mu_25ns/output_WplusH_HToZZTo4L_M125_13TeV_powheg2-minlo-HWJ_JHUgenV6_pythia8.root')
('Pr

In [4]:
#clean outliers
krm = []
for ik in events:
    nevents = len(events[ik])
    print '%s events: %i' % (ik,nevents)
    if(nevents < 10):
        krm.append(ik)
    #if(ik != 'VBF' and ik != 'qqZZ'):
    #    krm.append(ik)
        
for ik in krm:
    del events[ik]
keys = []
for ik in events:
    keys.append(ik)
print '----------- KEYS -----------'
print keys    

TTZ events: 183
DYJets events: 0
ZH events: 10135
VBF events: 38623
qqZZ events: 214
WZ events: 2
WH events: 21967
WW events: 0
WJets events: 0
ttH events: 17486
TTbar events: 2
TTWJets events: 3
WZZ events: 1
ggZZ events: 865
WWZ events: 0
GGH events: 16587
ZZZ events: 1
----------- KEYS -----------
['TTZ', 'ZH', 'VBF', 'qqZZ', 'WH', 'ttH', 'ggZZ', 'GGH']


In [5]:
sum = 0
for ik in events:
    sum += len(events[ik])
print 'Total events: %i' % sum

Total events: 106060


In [13]:
#organize events and splits data in 50% each
full_event_train = {}
full_event_test = {}
for ik in keys:
    full_event_train[ik] = []
    full_event_test[ik] = []
    
    
for ik in keys:
    for iev in range(len(events[ik])):
        #just to recover lepton E to complite p4 - there's uncertanty probably
        lp4 = ROOT.TLorentzVector()
        for il in range(4):
            mass = 0.
            if(abs(events[ik][iev][il][3]) == 13):
                mass = 0.106
            lp4.SetPtEtaPhiM(events[ik][iev][il][0],events[ik][iev][il][1],events[ik][iev][il][2],mass)
            events[ik][iev][il][3] = lp4.E()

        if(iev < len(events[ik])/2):
            full_event_train[ik].append([
                events[ik][iev][0],  #l1
                events[ik][iev][1],  #l2
                events[ik][iev][2],  #l3
                events[ik][iev][3],  #l4
                events[ik][iev][4],  #j1
                events[ik][iev][5],  #j2
                events[ik][iev][6],  #j3
                events[ik][iev][7],  #Djet
                events[ik][iev][8],  #MELA
                events[ik][iev][9],  #event weight                
                events[ik][iev][10], #Higgs mass
                events[ik][iev][11], #Z1 mass
                events[ik][iev][12]  #Z2 mass
            ])
            
        else:
            full_event_test[ik].append([
                events[ik][iev][0],  #l1
                events[ik][iev][1],  #l2
                events[ik][iev][2],  #l3
                events[ik][iev][3],  #l4
                events[ik][iev][4],  #j1
                events[ik][iev][5],  #j2
                events[ik][iev][6],  #j3
                events[ik][iev][7],  #Djet
                events[ik][iev][8],  #MELA
                events[ik][iev][9],  #event weight                
                events[ik][iev][10], #Higgs mass
                events[ik][iev][11], #Z1 mass
                events[ik][iev][12]  #Z2 mass
            ])
            
print 'Train set size: %i' % len(full_event_train)
print 'Test set size: %i' % len(full_event_test)

Train set size: 8
Test set size: 8


In [14]:
# Run classifier with cross-validation and plot ROC curves
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from scipy import interp

from keras.layers import LSTM, Bidirectional, Masking
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.optimizers import SGD, Adam


# Bidirectional LSTM Model
def build_bilstm_model(nparticles, nfeatures):
    # Headline input: meant to receive sequences of 200 floats
    # Note that we can name any layer by passing it a "name" argument.
    i = Input(shape=(nparticles, nfeatures,), name='main_input')
    # the masking layer will prevent the LSTM layer to consider the 0-padded jet values
    m = Masking()(i)

    # A LSTM will transform the vector sequence into a single vector,
    # containing information about the entire sequence
    # the Bidirectional will make the LSTM cell read the sequence from end to start and start to end at the same time
    m = Bidirectional( LSTM(12) ) (m)

    
    auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(m)
    model = Model(input=[i], output=[auxiliary_output])
    opt = Adam()
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model           

In [15]:
#to select how many jets to use 7: 4leptons + 3jets
nparticles = 4
features = {
    'pt' : None,
    'eta': None
    #'phi': None,
    #'e'  : None
}
nfeatures = len(features)
model = build_bilstm_model(nparticles, nfeatures)
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 4, 2)              0         
_________________________________________________________________
masking_3 (Masking)          (None, 4, 2)              0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 24)                1440      
_________________________________________________________________
aux_output (Dense)           (None, 1)                 25        
Total params: 1,465
Trainable params: 1,465
Non-trainable params: 0
_________________________________________________________________
None


/afs/cern.ch/user/m/mmelodea/keras/lib/python2.7/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[sigmoid.0..., inputs=[/main_inp...)`


In [16]:
#prepare train set
nsig = 0
nbkg = 0
for ik in keys:
    if(ik == 'VBF'):
        nsig = len(full_event_train['VBF'])
    else:
        nbkg += len(full_event_train[ik])
        
#print 'NSig: %i' % nsig
#print 'NBkg: %i' % nbkg

momentum_input_train = {}
momentum_input_train['signal'] = np.zeros((nsig, nparticles, nfeatures)) # 3 momentum
momentum_input_train['background'] = np.zeros((nbkg, nparticles, nfeatures)) # 3 momentum

weights = {}
weights['signal'] = []
weights['background'] = []

siev = 0
biev = 0
for ik in keys:
    if(ik == 'VBF'):
        for i in range(len(full_event_train[ik])):
            weights['signal'].append( full_event_train[ik][i][9] )
            for j in range(nparticles):
                for iprop in range(nfeatures):
                    if(full_event_train[ik][i][j][iprop] != -999.):
                        momentum_input_train['signal'][siev][j][iprop] = full_event_train[ik][i][j][iprop]
                    else:
                        break
            siev += 1
    else:
        for i in range(len(full_event_train[ik])):
            weights['background'].append( full_event_train[ik][i][9] )
            for j in range(nparticles):
                for iprop in range(nfeatures):
                    if(full_event_train[ik][i][j][iprop] != -999.):
                        momentum_input_train['background'][biev][j][iprop] = full_event_train[ik][i][j][iprop]
                    else:
                        break
            biev += 1
            
weights_train_set = np.concatenate([weights['signal'],weights['background']])
print 'siev: %i' % siev
print 'biev: %i' % biev

siev: 19311
biev: 33716


In [17]:
#prepare test set
nsig = 0
nbkg = 0
for ik in keys:
    if(ik == 'VBF'):
        nsig = len(full_event_test['VBF'])
    else:
        nbkg += len(full_event_test[ik])
        
print 'NSig: %i' % nsig
print 'NBkg: %i' % nbkg

momentum_input_test = {}
momentum_input_test['signal'] = np.zeros((nsig, nparticles, nfeatures)) # 3 momentum
momentum_input_test['background'] = np.zeros((nbkg, nparticles, nfeatures)) # 3 momentum

Ydjet = {}
Ydjet['signal'] = []
Ydjet['background'] = []

Ymela = {}
Ymela['signal'] = []
Ymela['background'] = []

weights = {}
weights['signal'] = []
weights['background'] = []

siev = 0
biev = 0
for ik in keys:
    if(ik == 'VBF'):
        for i in range(len(full_event_test[ik])):
            Ydjet['signal'].append( full_event_test[ik][i][7] )
            Ymela['signal'].append( full_event_test[ik][i][8] )
            weights['signal'].append( full_event_test[ik][i][9] )
            for j in range(nparticles):
                for iprop in range(nfeatures):
                    if(full_event_test[ik][i][j][iprop] != -999.):
                        momentum_input_test['signal'][siev][j][iprop] = full_event_test[ik][i][j][iprop]
                    else:
                        break
            siev += 1
    else:
        for i in range(len(full_event_test[ik])):
            Ydjet['background'].append( full_event_test[ik][i][7] )
            Ymela['background'].append( full_event_test[ik][i][8] )
            weights['background'].append( full_event_test[ik][i][9] )
            for j in range(nparticles):
                for iprop in range(nfeatures):
                    if(full_event_test[ik][i][j][iprop] != -999.):
                        momentum_input_test['background'][biev][j][iprop] = full_event_test[ik][i][j][iprop]
                    else:
                        break
            biev += 1
            
Ydjet_test_set = np.concatenate([Ydjet['signal'],Ydjet['background']])
Ymela_test_set = np.concatenate([Ymela['signal'],Ymela['background']])
weights_test_set = np.concatenate([weights['signal'],weights['background']])            
print 'siev: %i' % siev
print 'biev: %i' % biev

NSig: 19312
NBkg: 33721
siev: 19312
biev: 33721


In [18]:
#train the network
X = {}
Y = {}

X['train'] = np.concatenate([momentum_input_train['signal'], momentum_input_train['background']])
Ys = np.ones(momentum_input_train['signal'].shape[0])
Yb = np.zeros(momentum_input_train['background'].shape[0])
Y['train'] = np.concatenate([Ys, Yb])

X['test'] = np.concatenate([momentum_input_test['signal'], momentum_input_test['background']])
Ys = np.ones(momentum_input_test['signal'].shape[0])
Yb = np.zeros(momentum_input_test['background'].shape[0])
Y['test'] = np.concatenate([Ys, Yb])

early_stopping = EarlyStopping(monitor='val_loss', patience=5)
lstm_model = build_bilstm_model(nparticles, nfeatures)
history = lstm_model.fit(X['train'], Y['train'], 
                               validation_data=(X['test'], Y['test']), 
                               epochs=200, batch_size=128, 
                               verbose=2, callbacks=[early_stopping])

Train on 53027 samples, validate on 53033 samples
Epoch 1/200
4s - loss: 0.6563 - acc: 0.6358 - val_loss: 0.6556 - val_acc: 0.6358
Epoch 2/200
4s - loss: 0.6557 - acc: 0.6358 - val_loss: 0.6556 - val_acc: 0.6358
Epoch 3/200
4s - loss: 0.6554 - acc: 0.6358 - val_loss: 0.6553 - val_acc: 0.6358
Epoch 4/200
4s - loss: 0.6554 - acc: 0.6358 - val_loss: 0.6553 - val_acc: 0.6358
Epoch 5/200
4s - loss: 0.6552 - acc: 0.6358 - val_loss: 0.6556 - val_acc: 0.6358
Epoch 6/200
4s - loss: 0.6553 - acc: 0.6358 - val_loss: 0.6552 - val_acc: 0.6358
Epoch 7/200
4s - loss: 0.6551 - acc: 0.6358 - val_loss: 0.6552 - val_acc: 0.6358
Epoch 8/200
4s - loss: 0.6550 - acc: 0.6358 - val_loss: 0.6550 - val_acc: 0.6358
Epoch 9/200
4s - loss: 0.6550 - acc: 0.6358 - val_loss: 0.6550 - val_acc: 0.6358
Epoch 10/200
4s - loss: 0.6550 - acc: 0.6358 - val_loss: 0.6549 - val_acc: 0.6358
Epoch 11/200
4s - loss: 0.6550 - acc: 0.6358 - val_loss: 0.6549 - val_acc: 0.6358
Epoch 12/200
4s - loss: 0.6549 - acc: 0.6358 - val_loss: 

/afs/cern.ch/user/m/mmelodea/keras/lib/python2.7/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[sigmoid.0..., inputs=[/main_inp...)`


In [19]:
fig = pyp.figure(figsize=(10,10))
pyp.rc("font", size=20)

#train set unormalized roc
Y_score = lstm_model.predict(X['train'])
fpr, tpr, thresholds = roc_curve(Y['train'], Y_score)
roc_auc = auc(fpr, tpr)
pyp.plot(fpr, tpr, lw=2, color='blue', label='Train set unormalized ROC(area = %0.2f)' % roc_auc)

#train set normalized roc
fpr, tpr, thresholds = roc_curve(Y['train'], Y_score, sample_weight=weights_train_set)
roc_auc = auc(fpr, tpr)
pyp.plot(fpr, tpr, linestyle='--', lw=2, color='blue', label='Train set normalized ROC(area = %0.2f)' % roc_auc)


#test set unormalized roc
Y_score = lstm_model.predict(X['test'])
fpr, tpr, thresholds = roc_curve(Y['test'], Y_score)
roc_auc = auc(fpr, tpr)
pyp.plot(fpr, tpr, lw=2, color='red', label='Test set unormalized ROC(area = %0.2f)' % roc_auc)

#test set unormalized roc
fpr, tpr, thresholds = roc_curve(Y['test'], Y_score, sample_weight=weights_test_set)
roc_auc = auc(fpr, tpr)
pyp.plot(fpr, tpr, linestyle='--', lw=2, color='red', label='Test set normalized ROC(area = %0.2f)' % roc_auc)

#random
pyp.plot([0, 1], [0, 1], linestyle='--', lw=2, color='black', label='Luck')

pyp.xlim([0, 1.0])
pyp.ylim([0, 1.0])
pyp.xlabel('False Positive Rate')
pyp.ylabel('True Positive Rate')
pyp.title('Receiver Operating Characteristic')
pyp.legend(loc="lower right")
pyp.grid(True)
pyp.show()

ValueError: Reordering is not turned on, and the x array is not increasing: [  0.00000000e+00   5.26171107e-06   7.89426438e-05 ...,   9.99989835e-01
   9.99994776e-01   1.00000000e+00]

In [ ]:
fig = pyp.figure(figsize=(10,10))

val_loss = np.asarray(history.history['val_loss'])
loss = np.asarray(history.history['loss'])
pyp.plot(val_loss, label='loss validation 1')
pyp.plot(loss, label='loss train 1')
val_acc = np.asarray(history.history['val_acc'])
acc = np.asarray(history.history['acc'])
pyp.plot(val_acc, label='acc validation 1')
pyp.plot(acc, label='acc train 1')

pyp.legend()
pyp.xlabel('epoch')
pyp.ylim([0,1])
pyp.show()

In [ ]:
fig = pyp.figure(figsize=(10,10))

#unormalized rocs
fpr, tpr, thresholds = roc_curve(Y['test'], Ydjet_test_set)
roc_auc = auc(fpr, tpr)
print 'Djet ROC AUC: %.2f' % roc_auc
pyp.plot(fpr, tpr, lw=2, color='blue', label=('Djet unormalized ROC (area = %0.2f)' % roc_auc))

fpr, tpr, thresholds = roc_curve(Y['test'], Ymela_test_set)
roc_auc = auc(fpr, tpr)
print 'MELA ROC AUC: %.2f' % roc_auc
pyp.plot(fpr, tpr, lw=2, color='red', label=('MELA unormalized ROC (area = %0.2f)' % roc_auc))

Y_score = lstm_model.predict(X['test'])
fpr, tpr, thresholds = roc_curve(Y['test'], Y_score)
roc_auc = auc(fpr, tpr)
print 'LSTM ROC AUC: %.2f' % roc_auc
pyp.plot(fpr, tpr, lw=2, color='violet', label=('LSTM unormalized ROC (area = %0.2f)' % roc_auc))

#normalized rocs
fpr, tpr, thresholds = roc_curve(Y['test'], Ydjet_test_set, sample_weight=weights_test_set)
roc_auc = auc(fpr, tpr)
print 'Djet ROC AUC: %.2f' % roc_auc
pyp.plot(fpr, tpr, linestyle='--', lw=2, color='blue', label=('Djet normalized ROC (area = %0.2f)' % roc_auc))

fpr, tpr, thresholds = roc_curve(Y['test'], Ymela_test_set, sample_weight=weights_test_set)
roc_auc = auc(fpr, tpr)
print 'MELA ROC AUC: %.2f' % roc_auc
pyp.plot(fpr, tpr, linestyle='--', lw=2, color='red', label=('MELA normalized ROC (area = %0.2f)' % roc_auc))

Y_score = lstm_model.predict(X['test'])
fpr, tpr, thresholds = roc_curve(Y['test'], Y_score, sample_weight=weights_test_set)
roc_auc = auc(fpr, tpr)
print 'LSTM ROC AUC: %.2f' % roc_auc
pyp.plot(fpr, tpr, linestyle='--', lw=2, color='violet', label=('LSTM normalized ROC (area = %0.2f)' % roc_auc))


pyp.xlim([0, 1.0])
pyp.ylim([0, 1.0])
pyp.xlabel('False Positive Rate')
pyp.ylabel('True Positive Rate')
pyp.title('Receiver Operating Characteristic')
pyp.legend(loc="lower right")
pyp.grid(True)
pyp.show()

In [ ]:
#plot S/B ratio vs S eff
signal_sumw = 0
for iw in weights['signal']:
    signal_sumw += iw
print 'Normalized sig: %.2f' % signal_sumw

background_sumw = 0
for iw in weights['background']:
    background_sumw += iw
print 'Normalized bkg: %.2f' % background_sumw


djet_sb = []
djet_seff = []
mela_sb = []
mela_seff = []
lstm_sb = []
lstm_seff = []
djet_cuts = []
mela_cuts = []
lstm_cuts = []

full_sdjet = []
full_bdjet = []
full_smela = []
full_bmela = []
full_slstm = []
full_blstm = []
sweights = []
bweights = []

for cut in range(200):
    icut = cut/100.
    if(cut % 2 != 0):
        continue
    
    s_djet = 0
    s_mela = 0
    s_lstm = 0
    b_djet = 0
    b_mela = 0
    b_lstm = 0
    for i in range(len(Y['test'])):
        djet = Ydjet_test_set[i]
        mela = Ymela_test_set[i]
        lstm = Y_score[i][0]
        weight = weights_test_set[i]
        
        if(Y['test'][i] == 1):
            if(cut == 0):
                full_sdjet.append( djet )
                full_smela.append( mela )
                full_slstm.append( lstm )
                sweights.append( weight )
            
            if(djet > icut):
                s_djet += weight
            if(mela > icut):
                s_mela += weight
            if(lstm > icut):
                s_lstm += weight
        else:
            if(cut == 0):
                full_bdjet.append( djet )
                full_bmela.append( mela )
                full_blstm.append( lstm )
                bweights.append( weight )
            
            if(djet > icut):
                b_djet += weight
            if(mela > icut):
                b_mela += weight
            if(lstm > icut):
                b_lstm += weight
    
    if(b_djet > 0):
        djet_sb.append(s_djet/b_djet)
        djet_seff.append(s_djet/signal_sumw)
        djet_cuts.append(icut)
    
    if(b_mela > 0):
        mela_sb.append(s_mela/b_mela)
        mela_seff.append(s_mela/signal_sumw)
        mela_cuts.append(icut)
    
    if(b_lstm > 0):
        #print 'cut: %.3f, s_lstm: %.3f, b_lstm: %.3f, sb: %.3f' % (icut, s_lstm,b_lstm,s_lstm/b_lstm)
        lstm_sb.append(s_lstm/b_lstm)
        lstm_seff.append(s_lstm/signal_sumw)
        lstm_cuts.append(icut)

In [ ]:
fig = pyp.figure(figsize=(10,10))
bins = np.linspace(0, 3, 100)
pyp.hist(full_sdjet, bins, weights=sweights, alpha=0.5, color='red', label='s_Djet')
pyp.hist(full_bdjet, bins, weights=bweights, alpha=0.5, color='blue',label='b_Djet')
pyp.xlabel('Djet')
pyp.ylabel('Normalized')
pyp.ylim([0.001,10])
pyp.legend(loc='upper right')
pyp.gca().set_yscale("log")
pyp.show()

In [ ]:
fig = pyp.figure(figsize=(10,10))
bins = np.linspace(0, 1, 100)
pyp.hist(full_smela, bins, weights=sweights, alpha=0.5, color='red', label='s_MELA')
pyp.hist(full_bmela, bins, weights=bweights, alpha=0.5, color='blue',label='b_MELA')
pyp.xlabel('MELA')
pyp.ylabel('Normalized')
pyp.ylim([0.001,10])
pyp.legend(loc='upper right')
pyp.gca().set_yscale("log")
pyp.show()

In [ ]:
fig = pyp.figure(figsize=(10,10))
bins = np.linspace(0, 1, 100)
pyp.hist(full_slstm, bins, weights=sweights, alpha=0.5, color='red', label='s_LSTM')
pyp.hist(full_blstm, bins, weights=bweights, alpha=0.5, color='blue',label='b_LSTM')
pyp.xlabel('LSTM')
pyp.ylabel('Normalized')
pyp.ylim([0.0001,10])
pyp.legend(loc='upper right')
pyp.gca().set_yscale("log")
pyp.show()

In [ ]:
fig = pyp.figure(figsize=(10,10))
pyp.plot(djet_seff, djet_sb, lw=2, color='blue', label='Djet')
pyp.plot(mela_seff, mela_sb, lw=2, color='red', label='MELA')
pyp.plot(lstm_seff, lstm_sb, lw=2, color='black', label='LSTM')
pyp.xlim([0, 1])
pyp.xlabel('Seff')
pyp.ylim([0, 10])
pyp.ylabel('S/B')
pyp.legend(loc="upper right")
pyp.grid(True)
pyp.show()

In [ ]:
fig = pyp.figure(figsize=(10,10))
pyp.plot(djet_cuts, djet_sb, lw=2, color='blue', label='Djet')
pyp.plot(mela_cuts, mela_sb, lw=2, color='red', label='MELA')
pyp.plot(lstm_cuts, lstm_sb, lw=2, color='black', label='LSTM')
pyp.xlim([0, 3])
pyp.ylim([0, 10])
pyp.xlabel('Cut')
pyp.ylabel('S/B')
pyp.legend(loc="upper left")
pyp.grid(True)
pyp.show()

In [ ]:
fig = pyp.figure(figsize=(10,10))
pyp.plot(djet_cuts, djet_seff, lw=2, color='blue', label='Djet')
pyp.plot(mela_cuts, mela_seff, lw=2, color='red', label='MELA')
pyp.plot(lstm_cuts, lstm_seff, lw=2, color='black', label='LSTM')
pyp.xlim([0, 1])
pyp.ylim([0, 1])
pyp.xlabel('Cut')
pyp.ylabel('Signal eff')
pyp.legend(loc="upper right")
pyp.grid(True)

pyp.show()